In [1]:
import sklearn
import nltk

from generateWordFrequency import *
from naiveBayes import *
from fileWriteFunctions import *

In [2]:
AllClasses = ['story', 'ask_hn', 'show_hn', 'poll']
delta = 0.5
appendClassPrefix = 'prob_'

In [3]:
unfilteredTrainData = getDataframe(2018)
unfilteredTrainData = addTokenizedColumnofTitle(unfilteredTrainData)

In [4]:
unfilteredTestData = getDataframe(2019)
unfilteredTestData = addTokenizedColumnofTitle(unfilteredTestData)
unfilteredTestData = unfilteredTestData.reset_index()

In [5]:
filename = 'Stopwords.txt'
with open (filename, "r") as myfile:
    data = myfile.readlines()
filteredWordList = [word[0:-1] for word in data if len(word[0:-1])]

In [8]:
trainData = filterTokensByWordList(unfilteredTrainData.copy(),filteredWordList)
testData = filterTokensByWordList(unfilteredTestData.copy(),filteredWordList)

trainWords = getWordFrequencyDataframe(trainData,AllClasses)
priorProbabilities = getPriorProbabilities(trainData) 

trainWords = getWordFrequencyDataframe(trainData,AllClasses)
trainWords = obtainDataframeWithClassProbabilities(trainWords, AllClasses, delta, appendClassPrefix)
writeModel(trainWords,'stopword-model.txt',AllClasses,appendClassPrefix)
trainWords = renameModelRows(trainWords, AllClasses, appendClassPrefix)
model = [trainWords, priorProbabilities]

testData = generateCondClassProb(testData, model)
testData['predicted'] = generatePrediction(testData,AllClasses)
testResults = comparePredictions(testData,AllClasses)
writeDataframe(testResults,'stopword-result.txt')
check = testResults.comparision.value_counts()
accuracy = check[True]/(check[True]+check[False])

In [19]:
print(accuracy)
df_confusion = pd.crosstab(testResults['Post Type'], testResults['predicted'])
print(df_confusion)

0.9898480486827242
predicted  ask_hn  show_hn   story
Post Type                         
ask_hn       5422        4      28
poll            1        0       5
show_hn        58     4720     125
story         421      751  125680
